In [5]:
import earthaccess
# Step 1: Login using your Earthdata credentials
earthaccess.login()
print("hello")

Enter your Earthdata Login username:  devanandhpr
Enter your Earthdata password:  ········


hello


In [ ]:
import datetime
import os

# --- 1. Define Search Parameters ---

# Define the 3-year time range for data collection
# Adjust the start date to your actual required period. PACE data starts early 2024.
START_DATE = "2023-01-01T00:00:00Z"
END_DATE = "2025-12-31T23:59:59Z"

# Define your Area of Interest (AOI) as a Bounding Box: (lon_min, lat_min, lon_max, lat_max)
# !!! IMPORTANT: REPLACE THIS EXAMPLE BOX with the coordinates of your target ocean region.
# Example: Western Atlantic/Gulf Stream (-85 W to -50 W, 20 N to 45 N)
BOUNDING_BOX = (-85.0, 20.0, -50.0, 45.0)

# Directory where all the downloaded data will be saved
DOWNLOAD_DIR = "nasa_shark_data"

# Ensure the download directory exists
os.makedirs(DOWNLOAD_DIR, exist_ok=True)


# --- 2. Define NASA Data Collections (Short Names) ---

# These are the official Collection Short Names for Level 3/4 Gridded Data
# Note: These products contain latitude and longitude coordinates.
DATA_COLLECTIONS = {
    # 1. Primary Productivity & Chlorophyll (PACE - High Resolution, Species Type)
    "PACE_CHL": "PACE_OCI_L3M_CHL_V3",  # Level 3 Gridded Chlorophyll-a concentration
    "PACE_NPP": "PACE_OCI_L4_NPP_V1",    # Level 4 Gridded Net Primary Production
    # Note on Phytoplankton Species: Specific L3 products for community composition
    # (like Phytoplankton Functional Type - PFT) may be available by searching the OB.DAAC catalog.
    # For now, NPP and CHL are the main proxies.

    # 2. Historical Chlorophyll (MODIS-Aqua - For Long-Term Context)
    "MODIS_CHL": "MODISA_L3m_CHL_4km_MONTHLY", # Monthly 4km L3 Gridded Chlorophyll-a

    # 3. Ocean Dynamics (SWOT - Sea Surface Height/Eddy Tracking)
    "SWOT_SSH": "SWOT_L2_LR_SSH_EXPERT_2.0", # Sea Surface Height (SSH) - use for eddy tracking/currents

    # 4. Ancillary Data (Sea Surface Temperature - SST)
    # Using a common, gridded (L3) high-resolution global SST product
    "SST": "MUR-JPL-L4-GLOB-v4.1" # Example L4 SST product (often used as proxy for L3/L4)
}


# --- 3. Bulk Search and Download ---

print(f"Step 2: Searching for granules across {len(DATA_COLLECTIONS)} missions...")

all_results = []
for mission_name, short_name in DATA_COLLECTIONS.items():
    print(f"  -> Searching for {mission_name} ({short_name})...")

    # Search the Common Metadata Repository (CMR)
    results = earthaccess.search_data(
        short_name=short_name,
        temporal=(START_DATE, END_DATE),
        bounding_box=BOUNDING_BOX,
        cloud_hosted=True # Prioritize cloud-hosted data for fast access
    )

    if results:
        print(f"     Found {len(results)} granules for {mission_name}.")
        all_results.extend(results)
    else:
        print(f"     No granules found for {mission_name} in the specified time/area. Check dates/AOI.")


print(f"\nStep 3: Initiating bulk download of {len(all_results)} files...")
if all_results:
    # This downloads all files to subfolders within the DOWNLOAD_DIR
    earthaccess.download(all_results, DOWNLOAD_DIR)
    print(f"\n✅ Download complete! All files saved to the '{DOWNLOAD_DIR}' folder.")
else:
    print("❌ No data found across all searches. The download step was skipped.")

Step 2: Searching for granules across 5 missions...
  -> Searching for PACE_CHL (PACE_OCI_L3M_CHL_V3)...
     No granules found for PACE_CHL in the specified time/area. Check dates/AOI.
  -> Searching for PACE_NPP (PACE_OCI_L4_NPP_V1)...
     No granules found for PACE_NPP in the specified time/area. Check dates/AOI.
  -> Searching for MODIS_CHL (MODISA_L3m_CHL_4km_MONTHLY)...
     No granules found for MODIS_CHL in the specified time/area. Check dates/AOI.
  -> Searching for SWOT_SSH (SWOT_L2_LR_SSH_EXPERT_2.0)...
     Found 2438 granules for SWOT_SSH.
  -> Searching for SST (MUR-JPL-L4-GLOB-v4.1)...
     Found 1007 granules for SST.

Step 3: Initiating bulk download of 3445 files...


QUEUEING TASKS | : 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3445/3445 [00:00<00:00, 7675.86it/s]
PROCESSING TASKS | :   0%|                                                                                                                                                    | 1/3445 [00:53<51:38:07, 53.97s/it]
Error while downloading the file SWOT_L2_LR_SSH_Expert_001_285_20230731T090541_20230731T095612_PGC0_01.nc
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\sharkenv\lib\site-packages\urllib3\connectionpool.py", line 787, in urlopen
    response = self._make_request(
  File "C:\ProgramData\Anaconda3\envs\sharkenv\lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "C:\ProgramData\Anaconda3\envs\sharkenv\lib\site-packages\urllib3\connection.py", line 565, in getresponse
    httplib_response

In [ ]:
import earthaccess
import datetime
import os

# --- 1. Define Search Parameters ---

# Set a flexible time range. 
# MODIS/SWOT cover 2023+. PACE data is only available from March 2024 onwards.
START_DATE = "2023-01-01T00:00:00Z" # Keeps the full 3-year window for MODIS/SWOT
END_DATE = "2025-12-31T23:59:59Z"

# !!! IMPORTANT: VERIFY AND ADJUST YOUR AREA OF INTEREST (AOI) !!!
# Example: Western Atlantic/Gulf Stream (-85 W to -50 W, 20 N to 45 N)
BOUNDING_BOX = (-85.0, 20.0, -50.0, 45.0)

DOWNLOAD_DIR = "nasa_shark_data_revised_2"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)


# --- 2. Define REVISED NASA Data Collections (Short Names) ---

DATA_COLLECTIONS = {
    # *** RECOVERY MISSION 1: PACE (Chlorophyll & NPP) ***
    # PACE L3/L4 products start availability in March 2024. 
    # We will search the full time window; the CMR will only return 2024+ files.
    "PACE_CHL": "PACE_OCI_L3M_CHL_V3",  # High-res, new sensor
    "PACE_NPP": "PACE_OCI_L4_NPP_V1",    # Level 4 data product
    
    # *** RECOVERY MISSION 2: MODIS-Aqua (Historical Chlorophyll) ***
    # The short name 'MODISA_L3m_CHL_4km_MONTHLY' is sometimes rejected.
    # We use the generic collection name and constrain to monthly, mapped, and 4km,
    # or a known working name if possible. We will try a different standard L3 name.
    "MODIS_CHL": "MODISA_L3m_CHL", # Generic L3 mapped Chlorophyll short name

    # Data already found: SWOT and SST (Keep these names)
    "SWOT_SSH": "SWOT_L2_LR_SSH_EXPERT_2.0", 
    "SST": "MUR-JPL-L4-GLOB-v4.1" 
}


# --- 3. Run Search and Download for ALL MISSIONS (including successful ones) ---

print("Step 2: Performing Data Search...")

all_results = []
for mission_name, short_name in DATA_COLLECTIONS.items():
    print(f"\n  -> Searching for {mission_name} ({short_name}) from {START_DATE.split('T')[0]} to {END_DATE.split('T')[0]}...")

    # Search the Common Metadata Repository (CMR)
    # We add a constraint for Chlorophyll products to ensure monthly 4km resolution
    if mission_name == "MODIS_CHL":
        # Additional keywords to force monthly 4km selection for the MODIS generic collection
        keywords = ["Monthly", "4km"]
    else:
        keywords = []

    results = earthaccess.search_data(
        short_name=short_name,
        temporal=(START_DATE, END_DATE),
        bounding_box=BOUNDING_BOX,
        keywords=keywords,
        cloud_hosted=True
    )
    
    if results:
        print(f"     ✅ Found {len(results)} granules for {mission_name}.")
        all_results.extend(results)
    else:
        print(f"     ❌ FAILED: Found 0 granules for {mission_name}. Please verify product name and AOI.")


print(f"\nStep 3: Initiating bulk download of {len(all_results)} total files...")
if all_results:
    # This downloads all files to subfolders within the DOWNLOAD_DIR
    earthaccess.download(all_results, DOWNLOAD_DIR)
    print(f"\n✅ Download complete! All data saved to the '{DOWNLOAD_DIR}' folder.")
else:
    print("❌ No data found with the current parameters. Download skipped.")

#### trianing and playing with datas 

In [1]:
import xarray as xr
import pandas as pd
import glob
import os

DOWNLOAD_DIR = r"D:\Projects\hackaton\nasa\nasa_data" # Your download path

# Find all netCDF files
all_files = glob.glob(os.path.join(DOWNLOAD_DIR, '**', '*.nc'), recursive=True)

# 1. Load and Time-Align the data (e.g., to weekly averages)
def load_and_resample(files, variable_name, resample_rule='W'):
    try:
        # Open multiple files, handle different coordinates
        ds = xr.open_mfdataset(files, combine='by_coords', engine='netcdf4', parallel=True)
        
        # Select the target variable and rename it to a common 'feature' name
        # Also drop dimensions that are not Lat/Lon/Time (e.g., SWOT has extra dimensions)
        ds_resampled = ds[variable_name].mean(dim=ds[variable_name].dims).sel(time=slice(ds['time'].min(), ds['time'].max()))
        
        # Resample to weekly mean (W)
        return ds_resampled.resample(time=resample_rule).mean()
        
    except Exception as e:
        print(f"Error loading {variable_name}: {e}")
        return None

# --- Load and Process Each Feature ---
# NOTE: Variable names must match exactaly what's in the NetCDF files.

# 1. SST (L4) - Weekly Mean
sst_data = load_and_resample(glob.glob(os.path.join(DOWNLOAD_DIR, 'MUR-JPL-L4-GLOB-v4.1', '*.nc')), 'sst')

# 2. Chlorophyll (PACE) - Weekly Mean
pace_chl_data = load_and_resample(glob.glob(os.path.join(DOWNLOAD_DIR, 'PACE_OCI_L3M_CHL_NRT', '*.nc')), 'chlor_a')

# 3. MODIS CHL (Contextual) - Align to the same time period/grid later

# 4. SWOT SSH - Requires advanced processing (See 2.2)

Error loading sst: no files to open
Error loading chlor_a: no files to open


In [2]:
import xarray as xr
import glob
import os
import numpy as np

# --- ⚠️ SET YOUR TARGET DOWNLOAD DIRECTORY ---
# Assuming the latest folder holds all your successful downloads
DATA_DIR = r"D:\Your_Big_Drive\nasa_shark_data_recovery_final" # ADJUST PATH IF NECESSARY

# --- Load and Resample Function ---
def load_and_resample(short_name_part, variable_name, resample_rule='W-MON'):
    """Loads all files for a collection, extracts one variable, and resamples to weekly mean."""
    search_path = os.path.join(DATA_DIR, short_name_part, '**', '*.nc')
    files = glob.glob(search_path, recursive=True)
    if not files:
        print(f"Warning: No files found for {short_name_part}")
        return None
    
    # Open all files, trusting coordinates and combining along the time dimension
    ds = xr.open_mfdataset(files, combine='by_coords', engine='netcdf4', parallel=True)
    
    # Check if the variable exists and has valid coordinates (lat/lon/time)
    if variable_name not in ds:
        print(f"Error: Variable '{variable_name}' not found in {short_name_part} files.")
        return None
        
    # Isolate the variable and resample time to Weekly (starting Monday) mean
    # Note: SWOT L2 data may require selecting specific variables and dimensions, 
    # but we proceed with a generic structure for L3/L4 products here.
    return ds[variable_name].resample(time=resample_rule).mean()


# 1. Load All Features
sst = load_and_resample('*MUR-JPL-L4-GLOB*', 'sst')
pace_chl = load_and_resample('*PACE_OCI_L3M_CHL_NRT*', 'chlor_a')
pace_poc = load_and_resample('*PACE_OCI_L3M_POC_NRT*', 'poc')
swot_ssh = load_and_resample('*SWOT_L2_LR_SSH_EXPERT*', 'ssh_karin')

print("Data loaded. Starting feature engineering...")

Data loaded. Starting feature engineering...


In [5]:
import xarray as xr
import glob
import os
import numpy as np

# --- ⚠️ SET YOUR TARGET DOWNLOAD DIRECTORY ---
# Based on your file system, let's assume the files are stored inside
# D:\Projects\hackaton\nasa\nasa_shark_data_recovery_final\
# If 'nasa_shark_data_recovery_final' contains the short-name folders, 
# you need to find the full path to that folder and use it as DATA_DIR.

# Since you mentioned the previous error, let's target the latest folder:
DATA_DIR = r"D:\Projects\hackaton\nasa" 
# NOTE: If the data is actually in D:\Projects\hackaton\nasa\nasa_data, use that path!


# --- Load and Resample Function ---
def load_and_resample(short_name_part, variable_name, resample_rule='W-MON'):
    """Loads files by searching recursively within the DATA_DIR."""
    
    # *** KEY FIX in the search path: ***
    # This searches the DATA_DIR for any folder containing the 'short_name_part'
    # and finds all .nc files inside.
    search_path = os.path.join(DATA_DIR, short_name_part, '**', '*.nc')
    files = glob.glob(search_path, recursive=True)
    
    if not files:
        print(f"Warning: No files found for {short_name_part} in {DATA_DIR}")
        return None
        
    print(f"Loading {len(files)} files for {short_name_part}...")
    
    # Open all files, trusting coordinates and combining along the time dimension
    # Suppress warnings that often occur with complex netCDF structures
    ds = xr.open_mfdataset(files, combine='by_coords', engine='netcdf4', parallel=True, 
                           coords='minimal', data_vars='minimal', compat='override')
    
    # Check for variable existence
    if variable_name not in ds:
        print(f"Error: Variable '{variable_name}' not found in {short_name_part} files.")
        return None
        
    # Isolate and resample
    return ds[variable_name].resample(time=resample_rule).mean()


# 1. Load All Features
sst = load_and_resample('*MUR-JPL-L4-GLOB*', 'sst')
pace_chl = load_and_resample('*PACE_OCI_L3M_CHL_NRT*', 'chlor_a')
pace_poc = load_and_resample('*PACE_OCI_L3M_POC_NRT*', 'poc')
swot_ssh = load_and_resample('*SWOT_L2_LR_SSH_EXPERT*', 'ssh_karin')

print("Data loaded. Starting feature engineering...")


# --- 2. Feature Engineering and Model Setup (Continue from previous plan) ---

if pace_chl is None or sst is None or swot_ssh is None:
    print("FATAL ERROR: Core datasets are missing. Cannot proceed with modeling.")
else:
    # --- Feature Engineering: Create Eddy Proxy (SSH Gradient) ---
    print("\nStarting spatial alignment and gradient calculation...")
    
    # Regrid SST onto the PACE CHL grid (assuming PACE grid is the reference)
    # Using simple linear interpolation
    sst_regridded = sst.interp(
        lat=pace_chl.lat, 
        lon=pace_chl.lon, 
        method='linear'
    ).rename('SST_C')

    # Regrid SWOT SSH onto the PACE grid
    swot_ssh_regridded = swot_ssh.interp(
        lat=pace_chl.lat, 
        lon=pace_chl.lon, 
        method='linear'
    ).rename('SSH_m')

    # Calculate the SSH Gradient Magnitude (Feature for ocean dynamics)
    grad_y, grad_x = np.gradient(swot_ssh_regridded.values, axis=(1, 2))
    ssh_gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    
    ssh_grad_feature = swot_ssh_regridded.copy(data=ssh_gradient_magnitude)
    ssh_grad_feature.name = 'SSH_Grad_Magnitude'

    # 3. Create the Final Harmonized Feature Dataset (X)
    X_features = xr.Dataset({
        'SST': sst_regridded,
        'CHL': pace_chl.rename('CHL_mgm3'),
        'POC': pace_poc.rename('POC_mgm3'),
        'SSH_GRAD': ssh_grad_feature
    }).dropna(dim='time', how='all')

    print("\n✅ Final Feature Matrix (X) created and aligned.")
    # print(X_features) # Uncomment to inspect the final Xarray Dataset structure


    # --- 4. MODELING STEP ---
    print("\nStarting Model Training (Requires external Shark Data Y)...")
    
    # 1. Flatten X features into a time-series DataFrame (average over space)
    X_train_df = X_features.mean(dim=['lat', 'lon']).to_dataframe()
    X_train_df = X_train_df.drop(columns=['SSH_m']) 
    
    # 2. Define DUMMY Y TARGETS (Must be replaced by REAL shark data)
    from sklearn.model_selection import train_test_split
    from sklearn.multioutput import MultiOutputRegressor
    from xgboost import XGBRegressor
    import pandas as pd

    Y_TARGETS = pd.DataFrame({
        'Lon_min': np.random.uniform(-70, -60, size=len(X_train_df)),
        'Lat_min': np.random.uniform(25, 30, size=len(X_train_df)),
        'Lon_max': np.random.uniform(-55, -45, size=len(X_train_df)),
        'Lat_max': np.random.uniform(35, 40, size=len(X_train_df))
    }, index=X_train_df.index)

    X_final = X_train_df.dropna()
    Y_final = Y_TARGETS.loc[X_final.index].dropna() # Align and drop NaNs

    # Final alignment check
    X_data = X_final.loc[Y_final.index].values
    Y_data = Y_final.values

    if len(X_data) > 0:
        # Split Data
        X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, shuffle=False)

        # Train Model
        multi_output_model = MultiOutputRegressor(XGBRegressor(n_estimators=50, random_state=42))
        multi_output_model.fit(X_train, Y_train)

        # Predict the Bounding Box
        Weekly_Prediction_X = X_data[-1].reshape(1, -1) 
        Predicted_Box_Coords = multi_output_model.predict(Weekly_Prediction_X)[0]

        # Output the prediction
        print("\n🎯 **Predicted Shark Foraging Bounding Box (Dummy Y-Data):**")
        print(f"   Lon_min: {Predicted_Box_Coords[0]:.2f}")
        print(f"   Lat_min: {Predicted_Box_Coords[1]:.2f}")
        print(f"   Lon_max: {Predicted_Box_Coords[2]:.2f}")
        print(f"   Lat_max: {Predicted_Box_Coords[3]:.2f}")
    else:
        print("Model training failed: Not enough aligned data points for X and Y.")

Data loaded. Starting feature engineering...
FATAL ERROR: Core datasets are missing. Cannot proceed with modeling.


In [8]:
import os
import shutil
import glob

# --- ⚠️ CONFIGURATION ---
# Base path containing all your failed/successful download attempts
BASE_DOWNLOAD_DIR = r"D:\Projects\hackaton\nasa\nasa_shark_data" # Adjust this to your 'hackaton\nasa' directory

# Define the FINAL, CLEAN MASTER FOLDER
TRAINING_DATA_DIR = os.path.join(BASE_DOWNLOAD_DIR, "training_data")
os.makedirs(TRAINING_DATA_DIR, exist_ok=True)

# Define the desired clean folder names and their corresponding NASA short name components
MAPPING = {
    # Clean Name : Search Pattern
    "SST": '*GHRSST*',
    "PACE_CHL": '*PACE_OCI_L3M_CHL_NRT*',
    "PACE_POC": '*PACE_OCI_L3M_POC_NRT*',
    "SWOT_SSH": '*SWOT_L2_LR_SSH_EXPERT*',
    "MODIS_CHL": '*MODISA_L3M_CHL*', # This will cover different versions
}

# --- END CONFIGURATION ---

print(f"Starting file consolidation into: {TRAINING_DATA_DIR}")

# Step 1: Search all previous download attempt folders recursively
# This ensures we find data regardless of which attempt folder it ended up in.
previous_attempts = [
    'nasa_data', 'nasa_shark_data', 'nasa_shark_data_recovery', 
    'nasa_shark_data_recovery_final', 'nasa_shark_data_revised_2'
]

total_files_moved = 0

for clean_name, search_pattern in MAPPING.items():
    # Create the destination folder within the master training directory
    DEST_FOLDER = os.path.join(TRAINING_DATA_DIR, clean_name)
    os.makedirs(DEST_FOLDER, exist_ok=True)
    
    print(f"\nProcessing files for: {clean_name}")
    files_for_feature = 0

    # Search through all previous download folders
    for attempt_dir in previous_attempts:
        search_base = os.path.join(BASE_DOWNLOAD_DIR, attempt_dir)
        
        # Look for the NASA product folder (which contains the raw files)
        # We search for a folder name containing the search_pattern
        for product_folder in glob.glob(os.path.join(search_base, search_pattern + '*')):
            
            # Find all NetCDF files inside that product folder
            files_to_move = glob.glob(os.path.join(product_folder, '**', '*.nc'), recursive=True)
            
            for src_file in files_to_move:
                dest_file = os.path.join(DEST_FOLDER, os.path.basename(src_file))
                
                # Check if the file already exists in the destination to prevent copying duplicates
                if not os.path.exists(dest_file):
                    shutil.copy2(src_file, dest_file) # Use copy2 to preserve metadata
                    files_for_feature += 1
                
    print(f"  -> Moved/Copied {files_for_feature} files to {DEST_FOLDER}")
    total_files_moved += files_for_feature

print(f"\n✅ Consolidation complete. {total_files_moved} unique files moved into 'training_data' folder.")

Starting file consolidation into: D:\Projects\hackaton\nasa\nasa_shark_data\training_data

Processing files for: SST
  -> Moved/Copied 0 files to D:\Projects\hackaton\nasa\nasa_shark_data\training_data\SST

Processing files for: PACE_CHL
  -> Moved/Copied 0 files to D:\Projects\hackaton\nasa\nasa_shark_data\training_data\PACE_CHL

Processing files for: PACE_POC
  -> Moved/Copied 0 files to D:\Projects\hackaton\nasa\nasa_shark_data\training_data\PACE_POC

Processing files for: SWOT_SSH
  -> Moved/Copied 0 files to D:\Projects\hackaton\nasa\nasa_shark_data\training_data\SWOT_SSH

Processing files for: MODIS_CHL
  -> Moved/Copied 0 files to D:\Projects\hackaton\nasa\nasa_shark_data\training_data\MODIS_CHL

✅ Consolidation complete. 0 unique files moved into 'training_data' folder.


In [21]:
import os
import shutil
import glob
import time

# --- ⚠️ CONFIGURATION ---
# Base path containing the folder you want to clean
BASE_PATH = r"D:\Projects\hackaton\nasa\nasa_data"

# The source folder you want to move files OUT OF
SOURCE_DATA_FOLDER = os.path.join(BASE_PATH, "nasa_shark_data")

# Define the FINAL, CLEAN MASTER FOLDER
TRAINING_DATA_DIR = os.path.join(BASE_PATH, "training_data_clean")
os.makedirs(TRAINING_DATA_DIR, exist_ok=True)

# Define the desired clean feature names and their corresponding NASA short name components
# We use minimal prefixes to capture all versions/types within that product family.
MAPPING = {
    # Clean Folder Name : Minimal Search Prefix
    "SST": 'MUR-JPL', 
    "PACE_CHL": 'PACE_OCI_L3M_CHL',
    "PACE_POC": 'PACE_OCI_L3M_POC',
    "SWOT_SSH": 'SWOT_L2_LR_SSH',
    "MODIS_CHL": 'MODISA_L3M_CHL', 
}
# --- END CONFIGURATION ---

print(f"Starting final consolidation from '{SOURCE_DATA_FOLDER}' to '{TRAINING_DATA_DIR}'...")

total_files_moved = 0

for clean_name, search_prefix in MAPPING.items():
    
    DEST_FOLDER = os.path.join(TRAINING_DATA_DIR, clean_name)
    os.makedirs(DEST_FOLDER, exist_ok=True)
    
    print(f"\nProcessing files for: {clean_name}")
    files_for_feature = 0

    # Step 1: Look for any product folder inside the SOURCE_DATA_FOLDER that starts with the minimal prefix
    search_pattern = os.path.join(SOURCE_DATA_FOLDER, search_prefix + '*')
    
    # Step 2: Iterate over the found product folders (e.g., SWOT_L2_LR_SSH_EXPERT_2.0)
    for product_folder in glob.glob(search_pattern):
        
        # Step 3: Find all NetCDF files recursively inside the product folder
        files_to_move = glob.glob(os.path.join(product_folder, '**', '*.nc'), recursive=True)
        
        for src_file in files_to_move:
            dest_file = os.path.join(DEST_FOLDER, os.path.basename(src_file))
            
            # Copy only if it's not a duplicate (to be safe)
            if not os.path.exists(dest_file):
                try:
                    shutil.copy2(src_file, dest_file) # Use copy2 to preserve file metadata
                    files_for_feature += 1
                except Exception as e:
                    print(f"    ⚠️ Failed to copy {os.path.basename(src_file)}: {e}")
                    
    print(f"  -> Successfully moved/copied {files_for_feature} unique files to {DEST_FOLDER}")
    total_files_moved += files_for_feature

print(f"\n✅ FINAL CONSOLIDATION COMPLETE. {total_files_moved} unique files are organized in '{TRAINING_DATA_DIR}'.")


Starting final consolidation from 'D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data' to 'D:\Projects\hackaton\nasa\nasa_data\training_data_clean'...

Processing files for: SST
  -> Successfully moved/copied 0 unique files to D:\Projects\hackaton\nasa\nasa_data\training_data_clean\SST

Processing files for: PACE_CHL
  -> Successfully moved/copied 0 unique files to D:\Projects\hackaton\nasa\nasa_data\training_data_clean\PACE_CHL

Processing files for: PACE_POC
  -> Successfully moved/copied 0 unique files to D:\Projects\hackaton\nasa\nasa_data\training_data_clean\PACE_POC

Processing files for: SWOT_SSH
  -> Successfully moved/copied 0 unique files to D:\Projects\hackaton\nasa\nasa_data\training_data_clean\SWOT_SSH

Processing files for: MODIS_CHL
  -> Successfully moved/copied 0 unique files to D:\Projects\hackaton\nasa\nasa_data\training_data_clean\MODIS_CHL

✅ FINAL CONSOLIDATION COMPLETE. 0 unique files are organized in 'D:\Projects\hackaton\nasa\nasa_data\training_data_clean'.


In [30]:
import xarray as xr
import glob
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

# --- ⚠️ SET YOUR TARGET FOLDER (New Clean Location) ---
CLEAN_DATA_DIR = r"D:\Projects\hackaton\nasa\training_data_clean" 

# --- Load and Resample Function (Now simplified) ---
def load_and_resample_clean(feature_folder_name, variable_name, resample_rule='W-MON'):
    """Loads files from the cleanly organized subfolders."""
    
    search_path = os.path.join(CLEAN_DATA_DIR, feature_folder_name, '*.nc')
    files = glob.glob(search_path)
    
    if not files:
        # Check if the folder exists, indicating a problem before the file search
        if os.path.isdir(os.path.join(CLEAN_DATA_DIR, feature_folder_name)):
            print(f"Warning: Folder '{feature_folder_name}' is empty. No files to load.")
        else:
            print(f"Warning: Folder '{feature_folder_name}' not found.")
        return None
        
    print(f"Loading {len(files)} files for {feature_folder_name}...")
    
    try:
        # Open files. Use the simplified path now that files are moved.
        ds = xr.open_mfdataset(files, combine='by_coords', engine='netcdf4', parallel=True,
                               coords='minimal', data_vars='minimal', compat='override')
        
        if variable_name not in ds:
            print(f"Error: Variable '{variable_name}' not found.")
            return None
            
        return ds[variable_name].resample(time=resample_rule).mean()
        
    except Exception as e:
        print(f"Critical Error processing {feature_folder_name}: {e}")
        return None


# 1. Load All Features from the new clean folder structure
sst = load_and_resample_clean('SST', 'sst')
pace_chl = load_and_resample_clean('PACE_CHL', 'chlor_a')
pace_poc = load_and_resample_clean('PACE_POC', 'poc')
swot_ssh = load_and_resample_clean('SWOT_SSH', 'ssh_karin')
modis_chl = load_and_resample_clean('MODIS_CHL', 'chlor_a')

# ... (Continue with Feature Engineering and Model Training using the loaded xarray objects) ...

In [28]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import xarray as xr
import pandas as pd
import glob
import os

# --- Configuration (Change this path) ---
folder_path = r"D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data"
# ----------------------------------------

def process_nasa_shark_data(folder_path):
    """
    Reads all .nc files in the specified folder, combines their data,
    and returns a pandas DataFrame.
    """
    
    # 1. Find all NetCDF files
    nc_files = glob.glob(os.path.join(folder_path, "*.nc"))
    
    if not nc_files:
        print(f"🛑 Error: No '.nc' files found in the directory: {folder_path}")
        return pd.DataFrame() # Return an empty DataFrame

    print(f"✅ Found {len(nc_files)} NetCDF files. Starting to process...")

    # 2. Combine the datasets (optional, depending on file structure)
    # This step is highly dependent on the exact structure of your files.
    # Assuming they contain data for different time steps/locations
    try:
        # Open multiple files as a single dataset, potentially combining them along a time dimension
        datasets = [xr.open_dataset(f) for f in nc_files]
        combined_ds = xr.concat(datasets, dim='time', data_vars='minimal', coords='minimal', compat='override')
        
    except Exception as e:
        print(f"⚠️ Warning: Could not concatenate files. Processing them individually. Error: {e}")
        # If concatenation fails, we'll try to process the first file as an example.
        combined_ds = xr.open_dataset(nc_files[0])


    # 3. Convert xarray Dataset to a pandas DataFrame
    # .to_dataframe() converts the dataset, including coordinates (like lat, lon, time) 
    # and all variables, into a long-format pandas DataFrame.
    df = combined_ds.to_dataframe().reset_index()

    # 4. Display a summary of the resulting DataFrame
    print("\n" + "="*50)
    print("🦈 Resulting DataFrame (Head):")
    print("="*50)
    print(df.head())
    
    print("\n" + "="*50)
    print("📊 DataFrame Information:")
    print("="*50)
    df.info()
    
    return df

# Run the function
shark_data_df = process_nasa_shark_data(folder_path)

# You can now use 'shark_data_df' for your ML training.
# Example: print(shark_data_df.describe())

✅ Found 3204 NetCDF files. Starting to process...


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12240\1674362141.py:30: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  datasets = [xr.open_dataset(f) for f in nc_files]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12240\1674362141.py:30: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype att

⚠️ Warning: Could not concatenate files. Processing them individually. Error: [Errno -101] NetCDF: HDF error: 'D:\\Projects\\hackaton\\nasa\\nasa_data\\nasa_shark_data\\20230209090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'


In [ ]:
import xarray as xr
import pandas as pd
import glob
import os

# --- Configuration (Your specified path) ---
folder_path = r"D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data"
# ----------------------------------------

def process_nasa_shark_data_robust(folder_path):
    """
    Reads all .nc files in the specified folder using xarray's multi-file
    opening function and returns a pandas DataFrame.
    """
    
    # 1. Find all NetCDF files
    nc_files = glob.glob(os.path.join(folder_path, "*.nc"))
    
    if not nc_files:
        print(f"🛑 Error: No '.nc' files found in the directory: {folder_path}")
        return pd.DataFrame()

    print(f"✅ Found {len(nc_files)} NetCDF files. Starting robust processing...")

    # 2. Open and Combine the datasets
    try:
        # Use open_mfdataset to efficiently open and combine multiple files.
        # chunks={} loads the data lazily, which is memory efficient.
        # decode_timedelta=False explicitly silences the FutureWarning you encountered.
        combined_ds = xr.open_mfdataset(
            nc_files, 
            combine='by_coords',
            chunks={},
            decode_timedelta=False
        )
        print("✅ Datasets successfully combined.")
        
    except Exception as e:
        print("\n" + "="*50)
        print("🛑 FATAL ERROR DURING FILE OPENING/COMBINATION")
        print("="*50)
        print(f"The NetCDF files could not be opened/combined. The error was:")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Message: {e}")
        print("\nThis often means the files are corrupted or have inconsistent metadata.")
        return pd.DataFrame()

    # 3. Convert xarray Dataset to a pandas DataFrame
    # .to_dataframe() converts the dataset into a long-format pandas DataFrame.
    try:
        # Reset index moves coordinate variables (like time, lat, lon) from index to columns
        df = combined_ds.to_dataframe().reset_index()
    except Exception as e:
        print("\n" + "="*50)
        print("🛑 ERROR converting xarray Dataset to pandas DataFrame")
        print("="*50)
        print(f"Error: {e}")
        return pd.DataFrame()


    # 4. Display a summary of the resulting DataFrame
    print("\n" + "="*70)
    print("🦈 FINAL RESULT: Combined NASA Shark Habitat DataFrame (Head):")
    print("="*70)
    print(df.head())
    
    print("\n" + "="*70)
    print("📊 DataFrame Information:")
    print("="*70)
    df.info()
    
    return df

# Run the function
shark_data_df = process_nasa_shark_data_robust(folder_path)

In [1]:
import xarray as xr
import pandas as pd
import glob
import os

folder_path = r"D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data"

def process_nasa_shark_data_lazy(folder_path):
    # 1. Find all NetCDF files
    nc_files = glob.glob(os.path.join(folder_path, "*.nc"))
    
    if not nc_files:
        print(f"🛑 Error: No '.nc' files found in the directory: {folder_path}")
        return pd.DataFrame()

    print(f"✅ Found {len(nc_files)} NetCDF files. Starting lazy multi-file open...")

    try:
        # 2. Open all files using open_mfdataset (Multi-File Dataset)
        # This creates a 'virtual' combined dataset (Dask array) without loading data into RAM.
        combined_ds = xr.open_mfdataset(
            nc_files, 
            combine='by_coords',        # Combine along common dimensions (like time)
            chunks={'time': 1},         # Process one time step at a time (chunking)
            decode_timedelta=False      # Suppress the FutureWarning
        )
        print("✅ Datasets successfully loaded into a virtual Dask array (lazy).")
        
    except Exception as e:
        print(f"🛑 FATAL ERROR during lazy loading: {e}")
        return pd.DataFrame()

    # 3. Convert a small slice of the Dataset to a pandas DataFrame for a preview
    # NOTE: Do NOT call .to_dataframe() on the entire 3204 files! 
    # That would still load terabytes of data.

    # We take the first 10 time steps (files) and a small region of data for the head preview.
    # Replace this slicing with the specific data you need for ML training!
    preview_ds = combined_ds.sel(
        time=combined_ds.time[:10],   # First 10 days
        lat=slice(30, 35),            # Small latitude slice
        lon=slice(-80, -75)           # Small longitude slice
    ).load() # .load() executes the computation and brings the small slice into RAM
    
    df_preview = preview_ds.to_dataframe().reset_index()

    print("\n" + "="*70)
    print("🦈 PREVIEW DataFrame (Small Slice for Head/Info):")
    print("="*70)
    print(df_preview.head())
    print("\n" + "="*70)
    print("📊 PREVIEW DataFrame Information:")
    print("="*70)
    df_preview.info()
    
    return combined_ds  # Return the full xarray Dataset for ML work

# Run the function - this time, we return the xarray Dataset for further ML preparation
shark_data_ds = process_nasa_shark_data_lazy(folder_path)

if isinstance(shark_data_ds, xr.Dataset):
    print("\n\n✅ The full combined dataset is available as 'shark_data_ds' for ML.")
    print("Hint: Use slicing (.sel) on 'shark_data_ds' to extract data for your ML model.")

✅ Found 3204 NetCDF files. Starting lazy multi-file open...
🛑 FATAL ERROR during lazy loading: [Errno -101] NetCDF: HDF error: 'D:\\Projects\\hackaton\\nasa\\nasa_data\\nasa_shark_data\\20230209090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'


In [35]:
!pip install dask[array]
print("hello")

hello


In [2]:
import xarray as xr
import pandas as pd
import glob
import os
import warnings

# --- Configuration (Your specified path) ---
folder_path = r"D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data"
# ----------------------------------------

def is_netcdf_file_valid(file_path):
    """Checks if a single NetCDF file is readable without errors."""
    try:
        # Try to open the file using xarray and suppress warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # We only check the header/metadata, avoiding heavy data loading
            ds = xr.open_dataset(file_path, engine='netcdf4', decode_timedelta=False)
            ds.close() # Close immediately to free resources
        return True
    except Exception as e:
        # Catch any exceptions (like HDF Error) and treat the file as corrupted
        return False

def process_nasa_shark_data_filtered(folder_path):
    # 1. Find all NetCDF files
    all_nc_files = glob.glob(os.path.join(folder_path, "*.nc"))
    
    if not all_nc_files:
        print(f"🛑 Error: No '.nc' files found in the directory: {folder_path}")
        return None

    print(f"✅ Found {len(all_nc_files)} potential NetCDF files. Starting validation...")

    # 2. Validate and filter files
    valid_nc_files = []
    corrupted_count = 0
    
    # Check each file. This step might take a few minutes for 3000+ files.
    for i, file in enumerate(all_nc_files):
        if is_netcdf_file_valid(file):
            valid_nc_files.append(file)
        else:
            corrupted_count += 1
            print(f"🛑 Corrupted file found and skipped: {os.path.basename(file)}")
        
        # Simple progress update (optional)
        if (i + 1) % 100 == 0:
            print(f"   -> Progress: Checked {i + 1}/{len(all_nc_files)} files.")


    if not valid_nc_files:
        print(f"🛑 Fatal: All {len(all_nc_files)} files were corrupted.")
        return None
        
    print(f"\n✅ Validation complete. {len(valid_nc_files)} valid files remain. Skipped {corrupted_count} corrupted files.")
    
    # 3. Open and Combine the valid datasets using lazy loading
    try:
        combined_ds = xr.open_mfdataset(
            valid_nc_files, 
            combine='by_coords',        
            chunks={'time': 1},         
            decode_timedelta=False      
        )
        print("✅ Valid datasets successfully loaded into a virtual Dask array (lazy).")
        
    except Exception as e:
        print(f"🛑 FATAL ERROR during open_mfdataset: {e}")
        return None

    # 4. Create a small DataFrame preview
    try:
        # Select the first 10 time steps and a small slice of lat/lon for preview
        preview_ds = combined_ds.sel(
            time=combined_ds.time[:10],
            lat=slice(30, 35),            
            lon=slice(-80, -75)           
        ).load() 
        
        df_preview = preview_ds.to_dataframe().reset_index()

        print("\n" + "="*70)
        print("🦈 PREVIEW DataFrame (Small Slice for Head/Info):")
        print("="*70)
        print(df_preview.head())
        print("\n" + "="*70)
        print("📊 PREVIEW DataFrame Information:")
        print("="*70)
        df_preview.info()
        
    except Exception as e:
        print(f"⚠️ Warning: Could not generate DataFrame preview. Full dataset object returned. Error: {e}")
        
    return combined_ds

# Run the function
shark_data_ds = process_nasa_shark_data_filtered(folder_path)

if shark_data_ds is not None:
    print("\n\n✅ The full combined dataset is available as 'shark_data_ds'.")
    print("This object contains all features (SST, Ice, etc.) indexed by time, lat, and lon.")

✅ Found 3204 potential NetCDF files. Starting validation...
🛑 Corrupted file found and skipped: 20230209090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230210090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230213090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230214090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230215090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230216090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230330090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
   -> Progress: Checked 100/3204 files.
🛑 Corrupted file found and skipped: 20230413090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230415090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and 

In [6]:
import xarray as xr
import pandas as pd
import glob
import os
import warnings

# --- Configuration (Your specified path) ---
folder_path = r"D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data"
# ----------------------------------------

def is_netcdf_file_valid(file_path):
    """Checks if a single NetCDF file is readable without errors."""
    try:
        # Try to open the file using xarray and suppress warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # We only check the header/metadata, avoiding heavy data loading
            ds = xr.open_dataset(file_path, engine='netcdf4', decode_timedelta=False)
            ds.close() # Close immediately to free resources
        return True
    except Exception as e:
        # Catch any exceptions (like HDF Error) and treat the file as corrupted
        return False

def process_nasa_shark_data_filtered(folder_path):
    # 1. Find all NetCDF files
    all_nc_files = glob.glob(os.path.join(folder_path, "*.nc"))
    
    if not all_nc_files:
        print(f"🛑 Error: No '.nc' files found in the directory: {folder_path}")
        return None

    print(f"✅ Found {len(all_nc_files)} potential NetCDF files. Starting validation...")

    # 2. Validate and filter files
    valid_nc_files = []
    corrupted_count = 0
    df_preview = pd.DataFrame() # Initialize an empty DataFrame for the preview
    
    # Check each file. This step might take a few minutes for 3000+ files.
    for i, file in enumerate(all_nc_files):
        if is_netcdf_file_valid(file):
            valid_nc_files.append(file)
        else:
            corrupted_count += 1
            print(f"🛑 Corrupted file found and skipped: {os.path.basename(file)}")
        
        # Simple progress update (optional)
        if (i + 1) % 100 == 0:
            print(f"   -> Progress: Checked {i + 1}/{len(all_nc_files)} files.")


    if not valid_nc_files:
        print(f"🛑 Fatal: All {len(all_nc_files)} files were corrupted.")
        return None
        
    print(f"\n✅ Validation complete. {len(valid_nc_files)} valid files remain. Skipped {corrupted_count} corrupted files.")
    
    # 3. Open and Combine the valid datasets using lazy loading (REVISED FIX)
    try:
        # Sort the files chronologically using their filename
        # This is critical for combining by time
        valid_nc_files.sort()

        # Explicitly tell xarray to concatenate along the 'time' dimension.
        combined_ds = xr.open_mfdataset(
            valid_nc_files, 
            concat_dim='time',        # Explicitly concatenate along 'time'
            combine='nested',         # Use nested combination strategy to handle different file structures
            chunks={'time': 1},       
            decode_timedelta=False    
        )
        print("✅ Valid datasets successfully loaded into a virtual Dask array (lazy).")
        
    except Exception as e:
        print(f"🛑 FATAL ERROR during open_mfdataset: {e}")
        print("\n💡 Diagnostic Suggestion: The 'time' variable might be missing or non-standard across the valid files.")
        return None

    # 4. Create a small DataFrame preview
    try:
        # Select the first 10 time steps and a small slice of lat/lon for preview
        preview_ds = combined_ds.sel(
            time=combined_ds.time[:10],
            lat=slice(30, 35),            
            lon=slice(-80, -75)           
        ).load() 
        
        df_preview = preview_ds.to_dataframe().reset_index()

        print("\n" + "="*70)
        print("🦈 PREVIEW DataFrame (Small Slice for Head/Info):")
        print("="*70)
        print(df_preview.head())
        print("\n" + "="*70)
        print("📊 PREVIEW DataFrame Information:")
        print("="*70)
        df_preview.info()
        
    except Exception as e:
        print(f"⚠️ Warning: Could not generate DataFrame preview. Full dataset object returned. Error: {e}")
        
    # Return both the combined dataset and the DataFrame preview
    return combined_ds, df_preview 

# Run the function
result = process_nasa_shark_data_filtered(folder_path)

# Extract the dataset and the preview DataFrame
if result:
    shark_data_ds, shark_data_df_preview = result
    
    # Final confirmation and printing of the preview DataFrame
    if shark_data_ds is not None:
        print("\n\n✅ The full combined dataset is available as 'shark_data_ds'.")
        print("This object contains all features (SST, Ice, etc.) indexed by time, lat, and lon.")
        
        # Explicitly print the DataFrame if it was created
        if not shark_data_df_preview.empty:
            print("\n" + "="*70)
            print("FINAL REQUESTED DATAFRAME PRINT:")
            print("="*70)
            print(shark_data_df_preview.head(10)) # Print the first 10 rows for confirmation

✅ Found 3204 potential NetCDF files. Starting validation...
🛑 Corrupted file found and skipped: 20230209090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230210090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230213090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230214090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230215090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230216090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230330090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
   -> Progress: Checked 100/3204 files.
🛑 Corrupted file found and skipped: 20230413090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230415090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and 

Exception ignored in: <function CachingFileManager.__del__ at 0x000001465F457D90>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\sharkenv\lib\site-packages\xarray\backends\file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "C:\ProgramData\Anaconda3\envs\sharkenv\lib\site-packages\xarray\backends\file_manager.py", line 234, in close
    file.close()
KeyboardInterrupt: 
Exception ignored in: <function CachingFileManager.__del__ at 0x000001465F457D90>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\sharkenv\lib\site-packages\xarray\backends\file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "C:\ProgramData\Anaconda3\envs\sharkenv\lib\site-packages\xarray\backends\file_manager.py", line 234, in close
    file.close()
KeyboardInterrupt: 
Exception ignored in: <function CachingFileManager.__del__ at 0x000001465F457D90>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\

KeyboardInterrupt: 

In [ ]:
import xarray as xr
import pandas as pd
import glob
import os
import warnings

# --- Configuration (Your specified path) ---
folder_path = r"D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data"
# ----------------------------------------

def is_netcdf_file_valid(file_path):
    """Checks if a single NetCDF file is readable without errors."""
    try:
        # Try to open the file using xarray and suppress warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # We only check the header/metadata, avoiding heavy data loading
            ds = xr.open_dataset(file_path, engine='netcdf4', decode_timedelta=False)
            ds.close() # Close immediately to free resources
        return True
    except Exception as e:
        # Catch any exceptions (like HDF Error) and treat the file as corrupted
        return False

def process_nasa_shark_data_filtered(folder_path):
    # 1. Find all NetCDF files
    all_nc_files = glob.glob(os.path.join(folder_path, "*.nc"))
    
    if not all_nc_files:
        print(f"🛑 Error: No '.nc' files found in the directory: {folder_path}")
        return None

    print(f"✅ Found {len(all_nc_files)} potential NetCDF files. Starting validation...")

    # 2. Validate and filter files
    valid_nc_files = []
    corrupted_count = 0
    df_preview = pd.DataFrame() # Initialize an empty DataFrame for the preview
    
    # Check each file. This step might take a few minutes for 3000+ files.
    for i, file in enumerate(all_nc_files):
        if is_netcdf_file_valid(file):
            valid_nc_files.append(file)
        else:
            corrupted_count += 1
            print(f"🛑 Corrupted file found and skipped: {os.path.basename(file)}")
        
        # Simple progress update (optional)
        if (i + 1) % 100 == 0:
            print(f"   -> Progress: Checked {i + 1}/{len(all_nc_files)} files.")


    if not valid_nc_files:
        print(f"🛑 Fatal: All {len(all_nc_files)} files were corrupted.")
        return None
        
    print(f"\n✅ Validation complete. {len(valid_nc_files)} valid files remain. Skipped {corrupted_count} corrupted files.")
    
    # 3. Open and Combine the valid datasets using lazy loading (REVISED FIX)
    try:
        # Sort the files chronologically using their filename
        # This is critical for combining by time
        valid_nc_files.sort()

        # Explicitly tell xarray to concatenate along the 'time' dimension.
        combined_ds = xr.open_mfdataset(
            valid_nc_files, 
            concat_dim='time',        # Explicitly concatenate along 'time'
            combine='nested',         # Use nested combination strategy to handle different file structures
            chunks={'time': 1},       
            decode_timedelta=False    
        )
        print("✅ Valid datasets successfully loaded into a virtual Dask array (lazy).")
        
    except Exception as e:
        print(f"🛑 FATAL ERROR during open_mfdataset: {e}")
        print("\n💡 Diagnostic Suggestion: The 'time' variable might be missing or non-standard across the valid files.")
        return None

    # 4. Create a small DataFrame preview
    try:
        # Select the first 10 time steps and a small slice of lat/lon for preview
        preview_ds = combined_ds.sel(
            time=combined_ds.time[:10],
            lat=slice(30, 35),            
            lon=slice(-80, -75)           
        ).load() 
        
        df_preview = preview_ds.to_dataframe().reset_index()

        print("\n" + "="*70)
        print("🦈 PREVIEW DataFrame (Small Slice for Head/Info):")
        print("="*70)
        print(df_preview.head())
        print("\n" + "="*70)
        print("📊 PREVIEW DataFrame Information:")
        print("="*70)
        df_preview.info()
        
    except Exception as e:
        print(f"⚠️ Warning: Could not generate DataFrame preview. Full dataset object returned. Error: {e}")
        
    # Return both the combined dataset and the DataFrame preview
    return combined_ds, df_preview 

# Run the function
result = process_nasa_shark_data_filtered(folder_path)

# Extract the dataset and the preview DataFrame
if result:
    shark_data_ds, shark_data_df_preview = result
    
    # Final confirmation and printing of the preview DataFrame
    if shark_data_ds is not None:
        print("\n\n✅ The full combined dataset is available as 'shark_data_ds'.")
        print("This object contains all features (SST, Ice, etc.) indexed by time, lat, and lon.")
        
        # Explicitly print the DataFrame if it was created
        if not shark_data_df_preview.empty:
            print("\n" + "="*70)
            print("FINAL REQUESTED DATAFRAME PRINT:")
            print("="*70)
            print(shark_data_df_preview.head(10)) # Print the first 10 rows for confirmation

In [ ]:
import xarray as xr
import pandas as pd
import glob
import os
import warnings
# Ensure you have zarr and numcodecs installed: pip install zarr numcodecs

# --- Configuration (Your specified path) ---
folder_path = r"D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data"
# Define the path for the checkpoint file
checkpoint_path = os.path.join(folder_path, "combined_shark_features.zarr")
# ----------------------------------------

def is_netcdf_file_valid(file_path):
    # ... (Keep this function as is)
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            ds = xr.open_dataset(file_path, engine='netcdf4', decode_timedelta=False)
            ds.close()
        return True
    except Exception as e:
        return False

def process_nasa_shark_data_with_checkpoint(folder_path, checkpoint_path):
    """
    Checks for a checkpoint file. If found, loads it. 
    Otherwise, processes all individual NetCDF files and creates the checkpoint.
    """
    # 1. Check for Checkpoint
    if os.path.exists(checkpoint_path):
        print(f"✅ Checkpoint found! Loading combined dataset from: {checkpoint_path}")
        try:
            # Use chunks='auto' to load lazily, similar to open_mfdataset
            combined_ds = xr.open_zarr(checkpoint_path, chunks='auto')
            print("✅ Checkpoint loaded successfully (lazily). Skipping file validation.")
            return combined_ds, pd.DataFrame() # Return dummy DataFrame for compatibility
        except Exception as e:
            print(f"🛑 Error loading checkpoint: {e}. Attempting to re-process files.")
            os.remove(checkpoint_path) # Delete corrupted checkpoint

    # --- Start original file processing if checkpoint is not found/failed ---

    all_nc_files = glob.glob(os.path.join(folder_path, "*.nc"))
    
    if not all_nc_files:
        print(f"🛑 Error: No '.nc' files found in the directory: {folder_path}")
        return None

    print(f"✅ Found {len(all_nc_files)} potential NetCDF files. Starting validation...")

    # 2. Validate and filter files
    valid_nc_files = []
    corrupted_count = 0
    df_preview = pd.DataFrame()

    for i, file in enumerate(all_nc_files):
        if is_netcdf_file_valid(file):
            valid_nc_files.append(file)
        else:
            corrupted_count += 1
            print(f"🛑 Corrupted file found and skipped: {os.path.basename(file)}")
        if (i + 1) % 100 == 0:
            print(f"   -> Progress: Checked {i + 1}/{len(all_nc_files)} files.")

    if not valid_nc_files:
        print(f"🛑 Fatal: All {len(all_nc_files)} files were corrupted.")
        return None
        
    print(f"\n✅ Validation complete. {len(valid_nc_files)} valid files remain. Skipped {corrupted_count} corrupted files.")
    
    # 3. Open and Combine the valid datasets using lazy loading
    try:
        valid_nc_files.sort()
        combined_ds = xr.open_mfdataset(
            valid_nc_files, 
            concat_dim='time',        
            combine='nested',         
            chunks={'time': 1},       
            decode_timedelta=False    
        )
        print("✅ Valid datasets successfully loaded into a virtual Dask array (lazy).")
        
    except Exception as e:
        print(f"🛑 FATAL ERROR during open_mfdataset: {e}")
        return None
    
    # 4. CREATE CHECKPOINT (New Step)
    print(f"\n⏳ Creating checkpoint at: {checkpoint_path}. This may take a while...")
    try:
        # Save the combined dataset to Zarr. This forces evaluation of all Dask computations.
        combined_ds.to_zarr(checkpoint_path, mode='w')
        print("✅ Checkpoint successfully created!")
    except Exception as e:
        print(f"🛑 Checkpoint creation failed: {e}")


    # 5. Create a small DataFrame preview (This remains the same)
    try:
        preview_ds = combined_ds.sel(
            time=combined_ds.time[:10],
            lat=slice(30, 35),            
            lon=slice(-80, -75)           
        ).load() 
        
        df_preview = preview_ds.to_dataframe().reset_index()

        print("\n" + "="*70)
        print("🦈 PREVIEW DataFrame (Small Slice for Head/Info):")
        print("="*70)
        print(df_preview.head())
        print("\n" + "="*70)
        print("📊 PREVIEW DataFrame Information:")
        print("="*70)
        df_preview.info()
        
    except Exception as e:
        print(f"⚠️ Warning: Could not generate DataFrame preview. Full dataset object returned. Error: {e}")
        
    return combined_ds, df_preview 

# Run the function
result = process_nasa_shark_data_with_checkpoint(folder_path, checkpoint_path)

if result:
    shark_data_ds, shark_data_df_preview = result
    
    if shark_data_ds is not None:
        print("\n\n✅ The full combined dataset is available as 'shark_data_ds'.")
        print("Subsequent runs will load this data instantly from the checkpoint.")
        
        if not shark_data_df_preview.empty:
            print("\n" + "="*70)
            print("FINAL REQUESTED DATAFRAME PRINT:")
            print("="*70)
            print(shark_data_df_preview.head(10))

✅ Found 3204 potential NetCDF files. Starting validation...
🛑 Corrupted file found and skipped: 20230209090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230210090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230213090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230214090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230215090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230216090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230330090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
   -> Progress: Checked 100/3204 files.
🛑 Corrupted file found and skipped: 20230413090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and skipped: 20230415090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
🛑 Corrupted file found and 

In [3]:
import xarray as xr
import pandas as pd
import glob
import os
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
print("start")
# --- Configuration (Your specified path) ---
folder_path = r"D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data"
# ----------------------------------------

# NOTE: The previous script SUCCESSFULLY created the 'valid_nc_files' list.
# We will skip the time-consuming validation step IF you can access the 'valid_nc_files'
# list from your previous execution.

# If you need to re-run the whole process (recommended for safety):
def is_netcdf_file_valid(file_path):
    """Checks if a single NetCDF file is readable without errors."""
    try:
        with xr.open_dataset(file_path, engine='netcdf4', decode_timedelta=False) as ds:
            ds.close()
        return True
    except Exception:
        return False

def get_valid_files(folder_path):
    all_nc_files = glob.glob(os.path.join(folder_path, "*.nc"))
    print(f"Starting validation for {len(all_nc_files)} files...")
    valid_nc_files = [f for i, f in enumerate(all_nc_files) if is_netcdf_file_valid(f)]
    print(f"✅ Found {len(valid_nc_files)} valid files.")
    return valid_nc_files

# --- STEP 1: Load Valid Files Safely ---
# Run the validation to get the file list (or assume you have the list already)
valid_nc_files = get_valid_files(folder_path) 
valid_nc_files.sort() # Ensure chronological order

print("\n1. Combining valid files using 'by_coords' (safe method)...")

try:
    # Use the more general and safer 'by_coords' method for merging time series data
    shark_data_ds = xr.open_mfdataset(
        valid_nc_files, 
        combine='by_coords',        # Recommended method for geospatial time series
        chunks={'time': 10},        # Increase chunk size slightly to speed up computation
        decode_timedelta=False,     # Still suppressing the FutureWarning
        parallel=True               # Use Dask for parallel loading for faster metadata processing
    )
    print("✅ Datasets successfully loaded into a virtual Dask array.")

except Exception as e:
    print(f"🛑 FATAL ERROR during open_mfdataset (Final Attempt): {e}")
    print("Consider increasing your memory limit or processing in smaller batches.")
    shark_data_ds = None
    
# Check if the loading was successful
if shark_data_ds is None:
    print("\n🛑 Cannot proceed to ML training without the combined dataset.")
else:
    # ----------------------------------------------------------------------
    # --- STEP 2: ML DATA PREPARATION (Using the same simulation structure) ---
    # ----------------------------------------------------------------------

    print("\n2. Simulating shark observation data (REPLACE WITH YOUR REAL DATA)...")
    
    # Simulate a small dataset of shark observations
    num_obs = 1000
    shark_observations_df = pd.DataFrame({
        'time': pd.to_datetime('2023-01-01') + pd.to_timedelta(np.random.randint(0, 365, num_obs), unit='D'),
        'lat': np.random.uniform(30, 35, num_obs),  # Target region from preview slice
        'lon': np.random.uniform(-80, -75, num_obs),
        'shark_present': np.random.randint(0, 2, num_obs) 
    })
    
    # 2.1 Extract Features by Merging
    print("3. Extracting features at observation points...")
    
    # Round coordinates for nearest neighbor matching
    obs_df = shark_observations_df.copy()
    obs_df['lat_rounded'] = obs_df['lat'].round(2)
    obs_df['lon_rounded'] = obs_df['lon'].round(2)

    # Use xarray's nearest neighbor selection
    merged_features_ds = shark_data_ds.sel(
        time=obs_df.time, 
        lat=obs_df.lat_rounded, 
        lon=obs_df.lon_rounded, 
        method='nearest'
    )
    
    # Convert the selected features to a DataFrame
    training_df = merged_features_ds.to_dataframe().reset_index()
    training_df = training_df.merge(
        obs_df.drop(columns=['lat_rounded', 'lon_rounded']),
        on=['time'], how='inner'
    )
    training_df = training_df.dropna(subset=['analyzed_sst']) 

    print("4. Final Training Data Head:")
    print(training_df.head())
    
    # ----------------------------------------------------------------------
    # --- STEP 3: TRAIN XGBOOST CLASSIFIER ---
    # ----------------------------------------------------------------------
    
    print("\n5. Training XGBoost Classifier...")

    # Feature Engineering
    training_df['day_of_year'] = training_df['time'].dt.dayofyear
    training_df['month'] = training_df['time'].dt.month

    feature_cols = ['lat', 'lon', 'day_of_year', 'month', 'analyzed_sst', 'sea_ice_fraction']
    target_col = 'shark_present'

    X = training_df[feature_cols]
    Y = training_df[target_col]

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    xgb_model = XGBClassifier(
        objective='binary:logistic',
        n_estimators=100,
        learning_rate=0.1,
        use_label_encoder=False, 
        eval_metric='logloss',
        random_state=42
    )

    xgb_model.fit(X_train, Y_train)

    # ----------------------------------------------------------------------
    # --- STEP 4: EVALUATE MODEL ---
    # ----------------------------------------------------------------------

    Y_pred = xgb_model.predict(X_test)
    Y_proba = xgb_model.predict_proba(X_test)[:, 1]
    
    print("\n" + "="*50)
    print("🎯 FINAL XGBOOST MODEL RESULTS")
    print("="*50)

    print("\nClassification Report:")
    print(classification_report(Y_test, Y_pred, zero_division=0))

    auc_score = roc_auc_score(Y_test, Y_proba)
    print(f"ROC AUC Score: {auc_score:.4f}")
    
    print("\nFeature Importance:")
    importance = pd.Series(xgb_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
    print(importance)

start
Starting validation for 3204 files...


KeyboardInterrupt: 

In [ ]:
import xarray as xr
import pandas as pd
import glob
import os
import numpy as np
import time
import logging
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import warnings
print("start")
# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Set up logging for better debugging and monitoring progress
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configuration (Your specified path) ---
folder_path = r"D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data"
# ----------------------------------------

# --- Validation Function ---
def is_netcdf_file_valid(file_path):
    """Checks if a single NetCDF file is readable without errors."""
    try:
        with xr.open_dataset(file_path, engine='netcdf4', decode_timedelta=False) as ds:
            ds.close()
        return True
    except Exception as e:
        logging.error(f"Error reading file {file_path}: {e}")
        return False

def get_valid_files(folder_path):
    all_nc_files = glob.glob(os.path.join(folder_path, "*.nc"))
    logging.info(f"Starting validation for {len(all_nc_files)} files...")

    valid_nc_files = []
    start_time = time.time()
    for i, file in enumerate(all_nc_files):
        if is_netcdf_file_valid(file):
            valid_nc_files.append(file)
        if i % 100 == 0:  # Print progress every 100 files
            logging.info(f"Validated {i+1} files...")

    logging.info(f"Validation took {time.time() - start_time:.2f} seconds")
    logging.info(f"✅ Found {len(valid_nc_files)} valid files.")
    return valid_nc_files

# --- STEP 1: Load Valid Files Safely ---
# Run the validation to get the file list
valid_nc_files = get_valid_files(folder_path) 
valid_nc_files.sort()  # Ensure chronological order

logging.info("\n1. Combining valid files using 'by_coords' (safe method)...")

# --- Safely open NetCDF files with error handling ---
try:
    # Use the more general and safer 'by_coords' method for merging time series data
    shark_data_ds = xr.open_mfdataset(
        valid_nc_files, 
        combine='by_coords',        # Recommended method for geospatial time series
        chunks={'time': 10},         # Increase chunk size slightly to speed up computation
        decode_timedelta=False,      # Suppress warnings
        parallel=False               # Disable parallelism for simplicity
    )
    logging.info("✅ Datasets successfully loaded into a virtual Dask array.")
except Exception as e:
    logging.error(f"🛑 FATAL ERROR during open_mfdataset: {e}")
    shark_data_ds = None

# Check if the loading was successful
if shark_data_ds is None:
    logging.error("\n🛑 Cannot proceed to ML training without the combined dataset.")
else:
    # ----------------------------------------------------------------------
    # --- STEP 2: ML DATA PREPARATION (Using the same simulation structure) ---
    # ----------------------------------------------------------------------

    logging.info("\n2. Simulating shark observation data (REPLACE WITH YOUR REAL DATA)...")
    
    # Simulate a small dataset of shark observations
    num_obs = 1000
    shark_observations_df = pd.DataFrame({
        'time': pd.to_datetime('2023-01-01') + pd.to_timedelta(np.random.randint(0, 365, num_obs), unit='D'),
        'lat': np.random.uniform(30, 35, num_obs),  # Target region from preview slice
        'lon': np.random.uniform(-80, -75, num_obs),
        'shark_present': np.random.randint(0, 2, num_obs) 
    })
    
    # 2.1 Extract Features by Merging
    logging.info("3. Extracting features at observation points...")
    
    # Round coordinates for nearest neighbor matching
    obs_df = shark_observations_df.copy()
    obs_df['lat_rounded'] = obs_df['lat'].round(2)
    obs_df['lon_rounded'] = obs_df['lon'].round(2)

    # Use xarray's nearest neighbor selection
    merged_features_ds = shark_data_ds.sel(
        time=obs_df.time, 
        lat=obs_df.lat_rounded, 
        lon=obs_df.lon_rounded, 
        method='nearest'
    )
    
    # Convert the selected features to a DataFrame
    training_df = merged_features_ds.to_dataframe().reset_index()
    training_df = training_df.merge(
        obs_df.drop(columns=['lat_rounded', 'lon_rounded']),
        on=['time'], how='inner'
    )
    training_df = training_df.dropna(subset=['analyzed_sst'])  # Drop rows with missing 'analyzed_sst'

    logging.info("4. Final Training Data Head:")
    logging.info(f"{training_df.head()}")
    
    # ----------------------------------------------------------------------
    # --- STEP 3: TRAIN XGBOOST CLASSIFIER ---
    # ----------------------------------------------------------------------
    
    logging.info("\n5. Training XGBoost Classifier...")

    # Feature Engineering
    training_df['day_of_year'] = training_df['time'].dt.dayofyear
    training_df['month'] = training_df['time'].dt.month

    feature_cols = ['lat', 'lon', 'day_of_year', 'month', 'analyzed_sst', 'sea_ice_fraction']
    target_col = 'shark_present'

    # Check if 'sea_ice_fraction' column exists, if not, create it
    if 'sea_ice_fraction' not in training_df.columns:
        training_df['sea_ice_fraction'] = 0  # or np.nan, depending on how you'd handle missing data

    X = training_df[feature_cols]
    Y = training_df[target_col]

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    xgb_model = XGBClassifier(
        objective='binary:logistic',
        n_estimators=100,
        learning_rate=0.1,
        use_label_encoder=False, 
        eval_metric='logloss',
        random_state=42
    )

    xgb_model.fit(X_train, Y_train)

    # ----------------------------------------------------------------------
    # --- STEP 4: EVALUATE MODEL ---
    # ----------------------------------------------------------------------

    Y_pred = xgb_model.predict(X_test)
    Y_proba = xgb_model.predict_proba(X_test)[:, 1]
    
    logging.info("\n" + "="*50)
    logging.info("🎯 FINAL XGBOOST MODEL RESULTS")
    logging.info("="*50)

    logging.info("\nClassification Report:")
    logging.info(f"{classification_report(Y_test, Y_pred, zero_division=0)}")

    auc_score = roc_auc_score(Y_test, Y_proba)
    logging.info(f"ROC AUC Score: {auc_score:.4f}")
    
    logging.info("\nFeature Importance:")
    importance = pd.Series(xgb_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
    logging.info(f"{importance}")


2025-10-05 01:59:05,276 - INFO - Starting validation for 3204 files...
2025-10-05 01:59:06,091 - INFO - Validated 1 files...
2025-10-05 01:59:13,319 - ERROR - Error reading file D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data\20230209090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc: [Errno -101] NetCDF: HDF error: 'D:\\Projects\\hackaton\\nasa\\nasa_data\\nasa_shark_data\\20230209090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
2025-10-05 01:59:13,407 - ERROR - Error reading file D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data\20230210090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc: [Errno -101] NetCDF: HDF error: 'D:\\Projects\\hackaton\\nasa\\nasa_data\\nasa_shark_data\\20230210090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
2025-10-05 01:59:13,756 - ERROR - Error reading file D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data\20230213090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc: [Errno -101] NetCDF: HDF error: 'D:\\Projects\\hackaton\\nasa\\nasa_

In [ ]:
import xarray as xr
import pandas as pd
import glob
import os
import numpy as np
import time
import logging
from shapely.geometry import Polygon, Point
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Set up logging for better debugging and monitoring progress
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configuration (Your specified path) ---
folder_path = r"D:\Projects\hackaton\nasa\nasa_data\nasa_shark_data"
# ----------------------------------------

# --- Polygon Check Function ---
def is_point_in_polygon(lat, lon, polygon_coords):
    """Check if a given point (lat, lon) is inside the polygon."""
    polygon = Polygon(polygon_coords)
    point = Point(lon, lat)
    return polygon.contains(point)

# --- Season Filter Function ---
def filter_by_season(df, season):
    """Filter the DataFrame based on the season."""
    if season == 'summer':
        return df[df['time'].dt.month.isin([6, 7, 8])]
    elif season == 'winter':
        return df[df['time'].dt.month.isin([12, 1, 2])]
    else:
        logging.warning(f"Season {season} not recognized. Returning full dataset.")
        return df

# --- Validation and File Processing ---
def is_netcdf_file_valid(file_path):
    """Checks if a single NetCDF file is readable without errors."""
    try:
        with xr.open_dataset(file_path, engine='netcdf4', decode_timedelta=False) as ds:
            ds.close()
        return True
    except Exception as e:
        logging.error(f"Error reading file {file_path}: {e}")
        return False

def get_valid_files(folder_path):
    all_nc_files = glob.glob(os.path.join(folder_path, "*.nc"))
    logging.info(f"Starting validation for {len(all_nc_files)} files...")

    valid_nc_files = []
    start_time = time.time()
    for i, file in enumerate(all_nc_files):
        if is_netcdf_file_valid(file):
            valid_nc_files.append(file)
        if i % 100 == 0:  # Print progress every 100 files
            logging.info(f"Validated {i+1} files...")

    logging.info(f"Validation took {time.time() - start_time:.2f} seconds")
    logging.info(f"✅ Found {len(valid_nc_files)} valid files.")
    return valid_nc_files

# --- STEP 1: Load Valid Files Safely ---
# Run the validation to get the file list
valid_nc_files = get_valid_files(folder_path) 
valid_nc_files.sort()  # Ensure chronological order

logging.info("\n1. Combining valid files using 'by_coords' (safe method)...")

# --- Safely open NetCDF files with error handling ---
try:
    # Use the more general and safer 'by_coords' method for merging time series data
    shark_data_ds = xr.open_mfdataset(
        valid_nc_files, 
        combine='by_coords',        # Recommended method for geospatial time series
        chunks={'time': 10},         # Increase chunk size slightly to speed up computation
        decode_timedelta=False,      # Suppress warnings
        parallel=False               # Disable parallelism for simplicity
    )
    logging.info("✅ Datasets successfully loaded into a virtual Dask array.")
except Exception as e:
    logging.error(f"🛑 FATAL ERROR during open_mfdataset: {e}")
    shark_data_ds = None

# Check if the loading was successful
if shark_data_ds is None:
    logging.error("\n🛑 Cannot proceed to ML training without the combined dataset.")
else:
    # ----------------------------------------------------------------------
    # --- STEP 2: ML DATA PREPARATION (Using the same simulation structure) ---
    # ----------------------------------------------------------------------

    logging.info("\n2. Simulating shark observation data (REPLACE WITH YOUR REAL DATA)...")
    
    # Simulate a small dataset of shark observations
    num_obs = 1000
    shark_observations_df = pd.DataFrame({
        'time': pd.to_datetime('2023-01-01') + pd.to_timedelta(np.random.randint(0, 365, num_obs), unit='D'),
        'lat': np.random.uniform(30, 35, num_obs),  # Target region from preview slice
        'lon': np.random.uniform(-80, -75, num_obs),
        'shark_present': np.random.randint(0, 2, num_obs) 
    })
    
    # --- Step 2.1: Filter by Polygon Input ---
    polygon_coords = [(30, -80), (35, -80), (35, -75), (30, -75)]  # Example polygon (lat, lon)
    logging.info(f"Filtering data using polygon with coordinates: {polygon_coords}")

    shark_observations_df = shark_observations_df[
        shark_observations_df.apply(lambda row: is_point_in_polygon(row['lat'], row['lon'], polygon_coords), axis=1)
    ]
    logging.info(f"Filtered data to {len(shark_observations_df)} points inside the polygon.")
    
    # --- Step 2.2: Filter by Season Input ---
    season = 'summer'  # Example season input
    shark_observations_df = filter_by_season(shark_observations_df, season)
    logging.info(f"Filtered data to {len(shark_observations_df)} points for season: {season}")
    
    # 2.1 Extract Features by Merging
    logging.info("3. Extracting features at observation points...")
    
    # Round coordinates for nearest neighbor matching
    obs_df = shark_observations_df.copy()
    obs_df['lat_rounded'] = obs_df['lat'].round(2)
    obs_df['lon_rounded'] = obs_df['lon'].round(2)

    # Use xarray's nearest neighbor selection
    merged_features_ds = shark_data_ds.sel(
        time=obs_df.time, 
        lat=obs_df.lat_rounded, 
        lon=obs_df.lon_rounded, 
        method='nearest'
    )
    
    # Convert the selected features to a DataFrame
    training_df = merged_features_ds.to_dataframe().reset_index()
    training_df = training_df.merge(
        obs_df.drop(columns=['lat_rounded', 'lon_rounded']),
        on=['time'], how='inner'
    )
    training_df = training_df.dropna(subset=['analyzed_sst'])  # Drop rows with missing 'analyzed_sst'

    logging.info("4. Final Training Data Head:")
    logging.info(f"{training_df.head()}")
    
    # ----------------------------------------------------------------------
    # --- STEP 3: TRAIN XGBOOST CLASSIFIER ---
    # ----------------------------------------------------------------------
    
    logging.info("\n5. Training XGBoost Classifier...")

    # Feature Engineering
    training_df['day_of_year'] = training_df['time'].dt.dayofyear
    training_df['month'] = training_df['time'].dt.month

    feature_cols = ['lat', 'lon', 'day_of_year', 'month', 'analyzed_sst', 'sea_ice_fraction']
    target_col = 'shark_present'

    # Check if 'sea_ice_fraction' column exists, if not, create it
    if 'sea_ice_fraction' not in training_df.columns:
        training_df['sea_ice_fraction'] = 0  # or np.nan, depending on how you'd handle missing data

    X = training_df[feature_cols]
    Y = training_df[target_col]

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    xgb_model = XGBClassifier(
        objective='binary:logistic',
        n_estimators=100,
        learning_rate=0.1,
        use_label_encoder=False, 
        eval_metric='logloss',
        random_state=42
    )

    xgb_model.fit(X_train, Y_train)

    # ----------------------------------------------------------------------
    # --- STEP 4: EVALUATE MODEL ---
    # ----------------------------------------------------------------------

    Y_pred = xgb_model.predict(X_test)
   
